Source : [MIT 6.00.2x - Introduction to Computational Thinking and Data Science](https://www.edx.org/course/introduction-to-computational-thinking-and-data-4)

In [1]:
from lecture3_segment2 import *

## Graph Class 

### Common Representations of Digraphs

- Adjacency matrix
    - Rows : source nodes
    - Columns : destination nodes
    - Cell[s, d] = 1 if there is an edge from s to d
    - Cell[s, d] = 0 otherwise
                   
- <span style="color:red">Adjacency</span> list
    - Associate with each node a list of destination nodes

```Python
class Digraph(object):
    """edges is a dict mapping each node to a list of
    its children"""
    def __init__(self):
        self.edges = {}
    def addNode(self, node):
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.edges
    def getNode(self, name):
        for n in self.edges:
            if n.getName() == name:
                return n
        raise NameError(name)
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1] #omit final newline
```

```Python
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)
```

### A classic Graph Optimization Problem

- Shortest path from n1 to n2
    - shortest sequence of edges such that
        - Source node of first edge is n1
        - Destination of last edge is n2
        - For edges, e1 and e2, in the sequence, if e2 follows e1 in the sequence, the source of e2 is the destination of e1
- Shortest weighted path
    - Minimize the sum of the weights of the edges in the path

In [2]:
print(buildCityGraph(Graph))

Boston->Providence
Boston->New York
Boston->Providence
Boston->Los Angeles
Providence->Boston
Providence->Boston
Providence->New York
New York->Boston
New York->Providence
New York->Chicago
New York->Denver
Chicago->New York
Chicago->Denver
Denver->Chicago
Denver->Phoenix
Denver->New York
Phoenix->Denver
Los Angeles->Boston


In [3]:
print(buildCityGraph(Digraph))

Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
New York->Chicago
Chicago->Denver
Denver->Phoenix
Denver->New York
Los Angeles->Boston


### Exercise 2

Consider our representation of permutations of students in a line from Exercise 1. (The teacher only swaps the positions of two students that are next to each other in line.) Let's consider a line of three students, Alice, Bob, and Carol (denoted A, B, and C). Using the Graph class created in the lecture, we can create a graph with the design chosen in Exercise 1: vertices represent permutations of the students in line; edges connect two permutations if one can be made into the other by swapping two adjacent students.

We construct our graph by first adding the following nodes:
```python
nodes = []
nodes.append(Node("ABC")) # nodes[0]
nodes.append(Node("ACB")) # nodes[1]
nodes.append(Node("BAC")) # nodes[2]
nodes.append(Node("BCA")) # nodes[3]
nodes.append(Node("CAB")) # nodes[4]
nodes.append(Node("CBA")) # nodes[5]

g = Graph()
for n in nodes:
    g.addNode(n)
```

In [4]:
nodes = []
nodes.append(Node("ABC")) # nodes[0]
nodes.append(Node("ACB")) # nodes[1]
nodes.append(Node("BAC")) # nodes[2]
nodes.append(Node("BCA")) # nodes[3]
nodes.append(Node("CAB")) # nodes[4]
nodes.append(Node("CBA")) # nodes[5]

g = Graph()
for n in nodes:
    g.addNode(n)

g.addEdge(Edge(g.getNode('ABC'), g.getNode('BAC')))
g.addEdge(Edge(g.getNode('ABC'), g.getNode('ACB')))
g.addEdge(Edge(g.getNode('ACB'), g.getNode('CAB')))
g.addEdge(Edge(g.getNode('BAC'), g.getNode('BCA')))
g.addEdge(Edge(g.getNode('BCA'), g.getNode('CBA')))
g.addEdge(Edge(g.getNode('CAB'), g.getNode('CBA')))

In [5]:
for i in range(len(nodes)):
    print("edges from node{}".format(i))
    for e in g.childrenOf(nodes[i]):
        print(e)
    print(" ")

edges from node0
BAC
ACB
 
edges from node1
ABC
CAB
 
edges from node2
ABC
BCA
 
edges from node3
BAC
CBA
 
edges from node4
ACB
CBA
 
edges from node5
BCA
CAB
 


## Finding the Shortest Path

In [6]:
from lecture3_segment3 import *

### Depth-first Search (DFS)

- Similar to left-first depth-first method of enumerating a search tree
- Main difference is that graph might have cycles, so we must keep track of what nodes we have visited

#### Algorithm 1: Depth-frist Search (DFS)

```python
def DFS(graph, start, end, path, shortest, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph"""
    path = path + [start]
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest,
                              toPrint)
                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    return shortest
    
def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS(graph, start, end, [], None, toPrint)
```

In [7]:
testSP('Chicago', 'Boston', 'DFS')

Current DFS path: Chicago
Current DFS path: Chicago->Phoenix
Current DFS path: Chicago->Denver
Current DFS path: Chicago->Denver->Phoenix
Current DFS path: Chicago->Denver->New York
Already visited Chicago
There is no path from Chicago to Boston


In [8]:
testSP('Boston', 'Phoenix', 'DFS')

Current DFS path: Boston
Current DFS path: Boston->Providence
Already visited Boston
Current DFS path: Boston->Providence->New York
Current DFS path: Boston->Providence->New York->Chicago
Current DFS path: Boston->Providence->New York->Chicago->Phoenix
Current DFS path: Boston->Providence->New York->Chicago->Denver
Already visited New York
Current DFS path: Boston->New York
Current DFS path: Boston->New York->Chicago
Current DFS path: Boston->New York->Chicago->Phoenix
Current DFS path: Boston->New York->Chicago->Denver
Already visited New York
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix


#### Algorithm 2: Breadth-first Search (BFS)

```python
def BFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    initPath = [start]
    pathQueue = [initPath]
    while len(pathQueue) != 0:
        #Get and remove oldest element in pathQueue
        tmpPath = pathQueue.pop(0)
        if toPrint:
            print('Current BFS path:', printPath(tmpPath))
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None

def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return BFS(graph, start, end, toPrint)
```

In [10]:
testSP('Boston', 'Phoenix', 'DFS')

Current DFS path: Boston
Current DFS path: Boston->Providence
Already visited Boston
Current DFS path: Boston->Providence->New York
Current DFS path: Boston->Providence->New York->Chicago
Current DFS path: Boston->Providence->New York->Chicago->Phoenix
Current DFS path: Boston->Providence->New York->Chicago->Denver
Already visited New York
Current DFS path: Boston->New York
Current DFS path: Boston->New York->Chicago
Current DFS path: Boston->New York->Chicago->Phoenix
Current DFS path: Boston->New York->Chicago->Denver
Already visited New York
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix


In [11]:
testSP('Boston', 'Phoenix', 'BFS')

Current BFS path: Boston
Current BFS path: Boston->Providence
Current BFS path: Boston->New York
Current BFS path: Boston->Providence->New York
Current BFS path: Boston->New York->Chicago
Current BFS path: Boston->Providence->New York->Chicago
Current BFS path: Boston->New York->Chicago->Phoenix
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix


### Exercise 4

In [12]:
nodes = []
nodes.append(Node("ABC")) # nodes[0]
nodes.append(Node("ACB")) # nodes[1]
nodes.append(Node("BAC")) # nodes[2]
nodes.append(Node("BCA")) # nodes[3]
nodes.append(Node("CAB")) # nodes[4]
nodes.append(Node("CBA")) # nodes[5]

In [15]:
for e in nodes:
    print(e)
    
g = Graph()
for n in nodes:
    g.addNode(n)

ABC
ACB
BAC
BCA
CAB
CBA


In [16]:
g.addEdge(Edge(g.getNode('ABC'), g.getNode('ACB')))
g.addEdge(Edge(g.getNode('ABC'), g.getNode('BAC')))
g.addEdge(Edge(g.getNode('ACB'), g.getNode('CAB')))
g.addEdge(Edge(g.getNode('BAC'), g.getNode('BCA')))
g.addEdge(Edge(g.getNode('BCA'), g.getNode('CBA')))
g.addEdge(Edge(g.getNode('CAB'), g.getNode('CBA')))

In [32]:
priority = {
    'ABC':0,
    'ACB':1,
    'BAC':2,
    'BCA':3,
    'CAB':4,
    'CBA':5
}

def DFS_ex4(graph, start, end, path, shortest, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph"""
    path = path + [start]
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
#             if shortest == None or len(path[-1]) < len(shortest):
            if shortest == None or priority[path[0].getName()] < priority[path[-1].getName()]:
                newPath = DFS(graph, node, end, path, shortest,
                              toPrint)
                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    return shortest
    
def shortestPath_DFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS_ex4(graph, start, end, [], None, toPrint)

In [33]:
source = 'ABC'
destination = 'CAB'
sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination), toPrint=True)
if sp != None:
    print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
else:
    print('There is no path from', source, 'to', destination)

Current DFS path: ABC
Current DFS path: ABC->ACB
Already visited ABC
Current DFS path: ABC->ACB->CAB
Shortest path from ABC to CAB is ABC->ACB->CAB


In [34]:
source = 'CAB'
destination = 'ACB'
sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination), toPrint=True)
if sp != None:
    print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
else:
    print('There is no path from', source, 'to', destination)

Current DFS path: CAB
Current DFS path: CAB->ACB
Shortest path from CAB to ACB is CAB->ACB


In [35]:
source = 'BAC'
destination = 'CAB'
sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination), toPrint=True)
if sp != None:
    print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
else:
    print('There is no path from', source, 'to', destination)

Current DFS path: BAC
Current DFS path: BAC->ABC
Current DFS path: BAC->ABC->ACB
Already visited ABC
Current DFS path: BAC->ABC->ACB->CAB
Already visited BAC
Shortest path from BAC to CAB is BAC->ABC->ACB->CAB


In [36]:
source = 'BAC'
destination = 'BCA'
sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination), toPrint=True)
if sp != None:
    print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
else:
    print('There is no path from', source, 'to', destination)

Current DFS path: BAC
Current DFS path: BAC->ABC
Current DFS path: BAC->ABC->ACB
Already visited ABC
Current DFS path: BAC->ABC->ACB->CAB
Already visited ACB
Current DFS path: BAC->ABC->ACB->CAB->CBA
Current DFS path: BAC->ABC->ACB->CAB->CBA->BCA
Already visited CAB
Already visited BAC
Shortest path from BAC to BCA is BAC->ABC->ACB->CAB->CBA->BCA


In [37]:
source = 'BCA'
destination = 'ACB'
sp = shortestPath_DFS(g, g.getNode(source), g.getNode(destination), toPrint=True)
if sp != None:
    print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
else:
    print('There is no path from', source, 'to', destination)

Current DFS path: BCA
Current DFS path: BCA->BAC
Current DFS path: BCA->BAC->ABC
Current DFS path: BCA->BAC->ABC->ACB
Already visited BAC
Already visited BCA
Shortest path from BCA to ACB is BCA->BAC->ABC->ACB
